# Task 5: Backtesting the Optimized Portfolio
# 
 In this notebook, we simulate the performance of our recommended portfolio over the last year and compare it to a benchmark (60% SPY / 40% BND). We will:
 - Define a backtesting period.
 - Simulate portfolio returns using fixed weights.
 - Compare cumulative returns and Sharpe ratios to the benchmark.
 - Visualize and interpret the results.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

 ## 1. Load Data and Weights
# 
 We use the same price data as before and load the optimal weights from Task 4.


In [ ]:
# Load prices and weights (from previous notebook or CSV)
prices: DataFrame with columns ['TSLA', 'BND', 'SPY']
weights_df: DataFrame with columns ['Max Sharpe', 'Min Volatility']

# Example: Load from CSV if needed
prices = pd.read_csv('../data/all_prices.csv', index_col=0, parse_dates=True)
weights_df = pd.read_csv('../results/forecasting/optimal_portfolio_weights.csv', index_col=0)

# Use Max Sharpe weights for strategy
strategy_weights = weights_df['Max Sharpe'].values


## 2. Define Backtesting Period
# 
We use the last 252 trading days (approx. 1 year) for backtesting.


## Rolling Window Backtest

In [ ]:
indow = 252
strategy_cum_returns = []
benchmark_cum_returns = []
dates = []

for start in range(0, len(prices) - window):
    end = start + window
    window_returns = prices.iloc[start:end].pct_change().dropna()
    strat_ret = (window_returns * strategy_weights).sum(axis=1)
    bench_ret = (window_returns * benchmark_weights).sum(axis=1)
    strategy_cum_returns.append((1 + strat_ret).prod() - 1)
    benchmark_cum_returns.append((1 + bench_ret).prod() - 1)
    dates.append(prices.index[end-1])

plt.figure(figsize=(12,6))
plt.plot(dates, strategy_cum_returns, label='Strategy')
plt.plot(dates, benchmark_cum_returns, label='Benchmark')
plt.title('Rolling 1-Year Cumulative Returns')
plt.legend()
plt.show()

In [ ]:
strategy_returns = (backtest_returns * strategy_weights).sum(axis=1)
strategy_cum_returns = (1 + strategy_returns).cumprod()


# Transaction Costs

In [ ]:
transaction_cost = 0.001  # 0.1% per trade
turnover = np.abs(np.diff(strategy_weights)).sum()
strategy_returns_adj = strategy_returns - turnover * transaction_cost

## 4. Simulate Benchmark Portfolio
# 
The benchmark is a simple 60% SPY / 40% BND allocation.


In [ ]:
benchmark_weights = np.array([0, 0.4, 0.6])
benchmark_returns = (backtest_returns * benchmark_weights).sum(axis=1)
benchmark_cum_returns = (1 + benchmark_returns).cumprod()


## 5. Plot Cumulative Returns


In [ ]:
plt.figure(figsize=(12,6))
plt.plot(strategy_cum_returns, label='Strategy Portfolio')
plt.plot(benchmark_cum_returns, label='60% SPY / 40% BND Benchmark')
plt.title('Backtest: Cumulative Returns (Last 12 Months)')
plt.xlabel('Date')
plt.ylabel('Cumulative Return')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()


 ## 6. Calculate Sharpe Ratio and Total Return


In [ ]:
def sharpe_ratio(returns, risk_free_rate=0.0):
    return (returns.mean() * 252 - risk_free_rate) / (returns.std() * np.sqrt(252))

print("Strategy Sharpe Ratio:", sharpe_ratio(strategy_returns))
print("Benchmark Sharpe Ratio:", sharpe_ratio(benchmark_returns))
print("Strategy Total Return:", strategy_cum_returns.iloc[-1] - 1)
print("Benchmark Total Return:", benchmark_cum_returns.iloc[-1] - 1)


 ## 7. Summary and Interpretation
# 
 - If the strategy portfolio outperforms the benchmark in both total return and Sharpe ratio, it demonstrates the value of using forecast-driven optimization.
 - If not, discuss possible reasons (market regime change, model limitations, etc.).
 - Note that backtest results are based on historical data and do not guarantee future performance.


In [ ]:
if strategy_cum_returns.iloc[-1] > benchmark_cum_returns.iloc[-1]:
    print("\nThe optimized strategy outperformed the benchmark over the backtest period.")
else:
    print("\nThe benchmark outperformed the optimized strategy over the backtest period.")

print("Note: Backtest results are based on historical data and do not guarantee future performance.")


### **Conclusion**
# 
We have backtested our optimized portfolio and compared it to a traditional benchmark. This provides valuable insight into the real-world viability of our model-driven investment strategy.